In [11]:
import cv2
import numpy as np
import dlib
import pandas as pd
import os, glob
from imutils import face_utils
import argparse
import imutils

In [15]:
def extract_features (path):
    
    # Load the detector
    detector = dlib.get_frontal_face_detector()
    
    # Load the predictor
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
    emotion_codes = {
                    'Angry': 0,
                    'Disgust' : 1, 
                    'Fear' : 2, 
                    'Happy' : 3, 
                    'Sad' : 4, 
                    'Surprise' : 5, 
                    'Neutral': 6
                    }
    emotions_features = pd.DataFrame()
    
    for filepath in glob.glob(path):
        
        #Split a path to get emotion name
        emotion = filepath.split(os.sep)[1]
    
        # Match dictionary key format from folder name
        emotion_code = emotion_codes[emotion.capitalize()]
        
        
    # for filepath in glob.glob("Training_803556.jpg"):
        
        # Read the image
        img = cv2.imread(filepath)
    
        # Convert image into grayscale
        gray = cv2.cvtColor(src=img, code=cv2.COLOR_BGR2GRAY)

        # Use detector to find landmarks
        faces = detector(gray)
        
        for face in faces:
            x1 = face.left() # left point
            y1 = face.top() # top point
            x2 = face.right() # right point
            y2 = face.bottom() # bottom point


            landmarks = predictor(image=gray, box=face)
            
            emptyarray = []
            
            for n in range(0, 68):
                x = landmarks.part(n).x
                y = landmarks.part(n).y
    #             print(x)
                # Created a coordinate numpy array
                coordinates = np.array([x, y])
            
                # Append coordinate values (x, y) to emptyarray
                emptyarray.append(coordinates[0])
                emptyarray.append(coordinates[1])

                # Create dataframe
                df = pd.DataFrame({'emotion_code': emotion_code, 'emotion':emotion, 'features':[emptyarray]})
        
            # Append all df from all files/faces
            emotions_features = emotions_features.append(df)
        
    emotions_features['features']= emotions_features['features'].astype(str).str[1:-1]
    emotions_features2 = emotions_features['features'].str.split(",", n = 135, expand = True) 
    emotions_features2.rename(columns= lambda s:f"data{s}", inplace=True )

    emotions_features2 = emotions_features2.apply(pd.to_numeric)
    emotions_features2.fillna(0,inplace=True)
    emotions_features2 = emotions_features2.astype({"data0": int})

    emotions_features_final = pd.concat([emotions_features[['emotion_code','emotion']],emotions_features2], axis=1, join="inner")
    emotions_features_final.reset_index(drop=True, inplace=True)

    return emotions_features_final 

In [13]:
path = "emotions_dataset\\**\\*.jpg"
df = extract_features(path)
df.head()

,emotion_code,emotion,data0,data1,data2,data3,data4,data5,data6,data7,...,data126,data127,data128,data129,data130,data131,data132,data133,data134,data135
0,0,angry,68,77,64,96,62,115,62,133,...,144,156,153,157,143,155,138,154,131,153
1,0,angry,12,89,23,109,34,127,49,143,...,138,94,146,93,140,101,136,104,131,107
2,0,angry,18,76,22,98,29,119,37,139,...,113,126,130,133,114,143,106,144,97,144
3,0,angry,30,77,33,98,39,119,46,138,...,105,153,125,156,106,165,97,168,88,168
4,0,angry,22,111,27,129,32,147,37,164,...,112,157,131,157,114,159,106,161,97,162


In [16]:
def file_features(path):
    
    # Load the detector
    detector = dlib.get_frontal_face_detector()
    
    # Load the predictor
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    emotions_features = pd.DataFrame()
    
    for filepath in glob.glob(path):
        
        # Read the image
        img = cv2.imread(filepath)
    
        # Convert image into grayscale
        gray = cv2.cvtColor(src=img, code=cv2.COLOR_BGR2GRAY)

        # Use detector to find landmarks
        faces = detector(gray)
        
        for face in faces:
            x1 = face.left() # left point
            y1 = face.top() # top point
            x2 = face.right() # right point
            y2 = face.bottom() # bottom point


            landmarks = predictor(image=gray, box=face)
            
            emptyarray = []
            
            for n in range(0, 68):
                x = landmarks.part(n).x
                y = landmarks.part(n).y
    #             print(x)
                # Created a coordinate numpy array
                coordinates = np.array([x, y])
            
                # Append coordinate values (x, y) to emptyarray
                emptyarray.append(coordinates[0])
                emptyarray.append(coordinates[1])

                # Create dataframe
                df = pd.DataFrame({'features':[emptyarray]})
        
            # Append all df from all files/faces
            emotions_features = emotions_features.append(df)
        
    emotions_features['features']= emotions_features['features'].astype(str).str[1:-1]
    emotions_features2 = emotions_features['features'].str.split(",", n = 135, expand = True) 
    emotions_features2.rename(columns= lambda s:f"data{s}", inplace=True )

    emotions_features2 = emotions_features2.apply(pd.to_numeric)
    emotions_features2.fillna(0,inplace=True)
    emotions_features2 = emotions_features2.astype({"data0": int})

    
    emotions_features2.reset_index(drop=True, inplace=True)

    return emotions_features2

In [17]:
# Testing the file_features built in function
path = "face.png"
df2 = file_features(path)
df2

,data0,data1,data2,data3,data4,data5,data6,data7,data8,data9,...,data126,data127,data128,data129,data130,data131,data132,data133,data134,data135
0,260,159,259,177,261,195,264,213,270,228,...,337,226,352,221,337,227,330,228,322,228


# Data Pre-Processing

In [34]:
def process_data(df):
    X = df.drop(['emotion_code', 'emotion'], axis = 1)
    y = df['emotion']
    
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder, MinMaxScaler
    from tensorflow.keras.utils import to_categorical

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    
    X_scaler = MinMaxScaler().fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    
    # Step 1: Label-encode data set
    label_encoder = LabelEncoder()
    label_encoder.fit(y_train)
    encoded_y_train = label_encoder.transform(y_train)
    encoded_y_test = label_encoder.transform(y_test)
    
    # Step 2: Convert encoded labels to one-hot-encoding
    y_train_categorical = to_categorical(encoded_y_train)
    y_test_categorical = to_categorical(encoded_y_test)
    
    return X_train_scaled, y_train_categorical, X_test_scaled, y_test_categorical, y_test, label_encoder  

## Create a Deep Learning Model

In [50]:
def create_dl_model(X_train_scaled, y_train_categorical, X_test_scaled, y_test, label_encoder):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense
    from sklearn.preprocessing import LabelEncoder
    
    # Create model and add layers
    model = Sequential()
    model.add(Dense(units=200, activation='relu', input_dim=136))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=7, activation='softmax'))
    
    # Compile and fit the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    # Fit training data to model
    model.fit(
        X_train_scaled,
        y_train_categorical,
        epochs=600,
        shuffle=True,
        verbose=2
        )
    
    # Make Predictions
    encoded_predictions = model.predict_classes(X_test_scaled[:100])
#     prediction_labels = label_encoder.inverse_transform(encoded_predictions)
    
    result_df = pd.DataFrame()
    result_df['Predicted'] = prediction_labels
    result_df['Actual'] = list(y_test[:100])
    
    # Save model
    model.save("image_emotions_model_test.h5")
    
    return model, result_df

In [60]:
def make_prediction(model, path):
    
    # Extract image file features
    df = file_features(path)
    
    # Make prediction using imported model
    prediction = model.predict_classes(df)
    prediction_label = label_encoder.inverse_transform(prediction)
    
#     # Emotion labels
#     emotion_codes = {
#                 0: 'Angry',
#                 1:'Disgust' ,
#                 2:'Fear',
#                 3:'Happy',
#                 4:'Sad',
#                 5:'Surprise',
#                 6:'Neutral'
#                }
#     result = emotion_codes[prediction[0]]
    
#     return result    
    return prediction_label[0]

In [40]:
# Process my data extracted from image
X_train, y_train, X_test, y_test_categorical, y_test, label_encoder = process_data(df)

In [41]:
X_train.shape

(15852, 136)

In [42]:
# from tensorflow.keras.models import load_model
# model =load_model("image_emotions_model.h5")

# Create Model with processed data
model, results_df = create_dl_model(X_train, y_train, X_test, y_test, label_encoder)

Train on 15852 samples
Epoch 1/600
15852/15852 - 5s - loss: 1.7217 - accuracy: 0.3203
Epoch 2/600
15852/15852 - 2s - loss: 1.5748 - accuracy: 0.4013
Epoch 3/600
15852/15852 - 2s - loss: 1.5340 - accuracy: 0.4179
Epoch 4/600
15852/15852 - 2s - loss: 1.4678 - accuracy: 0.4447
Epoch 5/600
15852/15852 - 2s - loss: 1.4377 - accuracy: 0.4548
Epoch 6/600
15852/15852 - 2s - loss: 1.4305 - accuracy: 0.4559
Epoch 7/600
15852/15852 - 2s - loss: 1.4197 - accuracy: 0.4611
Epoch 8/600
15852/15852 - 2s - loss: 1.4214 - accuracy: 0.4633
Epoch 9/600
15852/15852 - 2s - loss: 1.4114 - accuracy: 0.4621
Epoch 10/600
15852/15852 - 2s - loss: 1.4016 - accuracy: 0.4631
Epoch 11/600
15852/15852 - 2s - loss: 1.4067 - accuracy: 0.4634
Epoch 12/600
15852/15852 - 2s - loss: 1.3960 - accuracy: 0.4686
Epoch 13/600
15852/15852 - 2s - loss: 1.3977 - accuracy: 0.4683
Epoch 14/600
15852/15852 - 2s - loss: 1.3880 - accuracy: 0.4693
Epoch 15/600
15852/15852 - 2s - loss: 1.3862 - accuracy: 0.4720
Epoch 16/600
15852/15852 -

Epoch 129/600
15852/15852 - 2s - loss: 0.9131 - accuracy: 0.6511
Epoch 130/600
15852/15852 - 2s - loss: 0.9272 - accuracy: 0.6448
Epoch 131/600
15852/15852 - 2s - loss: 0.8939 - accuracy: 0.6547
Epoch 132/600
15852/15852 - 2s - loss: 0.9056 - accuracy: 0.6517
Epoch 133/600
15852/15852 - 2s - loss: 0.8908 - accuracy: 0.6590
Epoch 134/600
15852/15852 - 2s - loss: 0.8742 - accuracy: 0.6659
Epoch 135/600
15852/15852 - 2s - loss: 0.9059 - accuracy: 0.6559
Epoch 136/600
15852/15852 - 2s - loss: 0.8867 - accuracy: 0.6617
Epoch 137/600
15852/15852 - 2s - loss: 0.8691 - accuracy: 0.6646
Epoch 138/600
15852/15852 - 2s - loss: 0.8700 - accuracy: 0.6668
Epoch 139/600
15852/15852 - 2s - loss: 0.8709 - accuracy: 0.6698
Epoch 140/600
15852/15852 - 2s - loss: 0.8711 - accuracy: 0.6677
Epoch 141/600
15852/15852 - 2s - loss: 0.8643 - accuracy: 0.6732
Epoch 142/600
15852/15852 - 2s - loss: 0.8584 - accuracy: 0.6746
Epoch 143/600
15852/15852 - 2s - loss: 0.8241 - accuracy: 0.6852
Epoch 144/600
15852/15852

15852/15852 - 2s - loss: 0.4778 - accuracy: 0.8232
Epoch 256/600
15852/15852 - 2s - loss: 0.4752 - accuracy: 0.8247
Epoch 257/600
15852/15852 - 2s - loss: 0.4992 - accuracy: 0.8182
Epoch 258/600
15852/15852 - 2s - loss: 0.4774 - accuracy: 0.8253
Epoch 259/600
15852/15852 - 2s - loss: 0.4796 - accuracy: 0.8265
Epoch 260/600
15852/15852 - 2s - loss: 0.4103 - accuracy: 0.8471
Epoch 261/600
15852/15852 - 2s - loss: 0.4565 - accuracy: 0.8339
Epoch 262/600
15852/15852 - 2s - loss: 0.4062 - accuracy: 0.8486
Epoch 263/600
15852/15852 - 2s - loss: 0.5045 - accuracy: 0.8181
Epoch 264/600
15852/15852 - 2s - loss: 0.4532 - accuracy: 0.8362
Epoch 265/600
15852/15852 - 2s - loss: 0.4361 - accuracy: 0.8368
Epoch 266/600
15852/15852 - 2s - loss: 0.4690 - accuracy: 0.8267
Epoch 267/600
15852/15852 - 2s - loss: 0.4382 - accuracy: 0.8389
Epoch 268/600
15852/15852 - 2s - loss: 0.4434 - accuracy: 0.8367
Epoch 269/600
15852/15852 - 2s - loss: 0.4264 - accuracy: 0.8422
Epoch 270/600
15852/15852 - 2s - loss: 

Epoch 382/600
15852/15852 - 2s - loss: 0.2955 - accuracy: 0.8966
Epoch 383/600
15852/15852 - 2s - loss: 0.2802 - accuracy: 0.9044
Epoch 384/600
15852/15852 - 2s - loss: 0.2968 - accuracy: 0.8982
Epoch 385/600
15852/15852 - 2s - loss: 0.3117 - accuracy: 0.8946
Epoch 386/600
15852/15852 - 2s - loss: 0.2416 - accuracy: 0.9176
Epoch 387/600
15852/15852 - 2s - loss: 0.2641 - accuracy: 0.9097
Epoch 388/600
15852/15852 - 2s - loss: 0.2950 - accuracy: 0.8997
Epoch 389/600
15852/15852 - 2s - loss: 0.2744 - accuracy: 0.9073
Epoch 390/600
15852/15852 - 2s - loss: 0.2282 - accuracy: 0.9223
Epoch 391/600
15852/15852 - 2s - loss: 0.2738 - accuracy: 0.9046
Epoch 392/600
15852/15852 - 2s - loss: 0.2775 - accuracy: 0.8982
Epoch 393/600
15852/15852 - 2s - loss: 0.3145 - accuracy: 0.8933
Epoch 394/600
15852/15852 - 2s - loss: 0.2618 - accuracy: 0.9111
Epoch 395/600
15852/15852 - 2s - loss: 0.2878 - accuracy: 0.8998
Epoch 396/600
15852/15852 - 2s - loss: 0.2247 - accuracy: 0.9217
Epoch 397/600
15852/15852

15852/15852 - 2s - loss: 0.2085 - accuracy: 0.9324
Epoch 509/600
15852/15852 - 2s - loss: 0.2282 - accuracy: 0.9232
Epoch 510/600
15852/15852 - 2s - loss: 0.2462 - accuracy: 0.9198
Epoch 511/600
15852/15852 - 2s - loss: 0.1917 - accuracy: 0.9401
Epoch 512/600
15852/15852 - 2s - loss: 0.1932 - accuracy: 0.9389
Epoch 513/600
15852/15852 - 2s - loss: 0.1858 - accuracy: 0.9372
Epoch 514/600
15852/15852 - 2s - loss: 0.2659 - accuracy: 0.9189
Epoch 515/600
15852/15852 - 2s - loss: 0.2316 - accuracy: 0.9236
Epoch 516/600
15852/15852 - 2s - loss: 0.2493 - accuracy: 0.9206
Epoch 517/600
15852/15852 - 2s - loss: 0.1780 - accuracy: 0.9430
Epoch 518/600
15852/15852 - 2s - loss: 0.1995 - accuracy: 0.9344
Epoch 519/600
15852/15852 - 2s - loss: 0.3216 - accuracy: 0.9006
Epoch 520/600
15852/15852 - 2s - loss: 1.2139 - accuracy: 0.5760
Epoch 521/600
15852/15852 - 2s - loss: 0.6372 - accuracy: 0.7771
Epoch 522/600
15852/15852 - 2s - loss: 0.2170 - accuracy: 0.9287
Epoch 523/600
15852/15852 - 2s - loss: 

In [59]:
# Make predictions
img_path = 'angry-man.jpg'
test_img = make_prediction(model, img_path)
test_img

'happy'